This exercise will require you to pull some data from the Qunadl API. Qaundl is currently the most widely used aggregator of financial market data.

As a first step, you will need to register a free account on the http://www.quandl.com website.

After you register, you will be provided with a unique API key, that you should store:

In [10]:
# Store the API key as a string - according to PEP8, constants are always named in all upper case
API_KEY = 'hdmMspfP8cLSSNxQB-5j'

Qaundl has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Quandl API instructions here: https://docs.quandl.com/docs/time-series

While there is a dedicated Python package for connecting to the Quandl API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/ ).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [33]:
# First, import the relevant modules

import requests
import json 
import operator
import numpy as np
import statistics 

In [12]:
# Now, call the Quandl API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned
url = 'https://www.quandl.com/api/v3/datasets/FSE/AFX_X.json?&start_date=2020-01-01&end_date=2020-01-02&api_key=' + API_KEY
r = requests.get(url)
sample_json = r.json()

In [13]:
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure
sample_json

{'dataset': {'id': 10095370,
  'dataset_code': 'AFX_X',
  'database_code': 'FSE',
  'name': 'Carl Zeiss Meditec (AFX_X)',
  'description': 'Stock Prices for Carl Zeiss Meditec (2020-11-02) from the Frankfurt Stock Exchange.<br><br>Trading System: Xetra<br><br>ISIN: DE0005313704',
  'refreshed_at': '2020-12-01T14:48:09.907Z',
  'newest_available_date': '2020-12-01',
  'oldest_available_date': '2000-06-07',
  'column_names': ['Date',
   'Open',
   'High',
   'Low',
   'Close',
   'Change',
   'Traded Volume',
   'Turnover',
   'Last Price of the Day',
   'Daily Traded Units',
   'Daily Turnover'],
  'frequency': 'daily',
  'type': 'Time Series',
  'premium': False,
  'limit': None,
  'transform': None,
  'column_index': None,
  'start_date': '2020-01-01',
  'end_date': '2020-01-02',
  'data': [['2020-01-02',
    114.0,
    114.9,
    112.7,
    113.8,
    None,
    143464.0,
    16291666.0,
    None,
    None,
    None]],
  'collapse': None,
  'order': None,
  'database_id': 6129}}

These are your tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

1) Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017

In [14]:
url = "https://www.quandl.com/api/v3/datasets/FSE/AFX_X.json?"+ "&start_date=2017-01-01&end_date=2017-12-31&api_key=" + API_KEY
r = requests.get(url)

In [15]:
r.status_code

200

2) Convert the returned JSON object into a Python dictionary.

In [16]:
afx_json = r.json()
type(afx_json)

dict

3) Calculate what the highest and lowest opening prices were for the stock in this period.

In [19]:
afx_json.keys()

dict_keys(['dataset'])

In [20]:
afx_json['dataset'].keys()

dict_keys(['id', 'dataset_code', 'database_code', 'name', 'description', 'refreshed_at', 'newest_available_date', 'oldest_available_date', 'column_names', 'frequency', 'type', 'premium', 'limit', 'transform', 'column_index', 'start_date', 'end_date', 'data', 'collapse', 'order', 'database_id'])

In [21]:
afx_json['dataset']['column_names']

['Date',
 'Open',
 'High',
 'Low',
 'Close',
 'Change',
 'Traded Volume',
 'Turnover',
 'Last Price of the Day',
 'Daily Traded Units',
 'Daily Turnover']

In [22]:
afx_json['dataset']['data'][0:2]

[['2017-12-29',
  51.76,
  51.94,
  51.45,
  51.76,
  None,
  34640.0,
  1792304.0,
  None,
  None,
  None],
 ['2017-12-28',
  51.65,
  51.82,
  51.43,
  51.6,
  None,
  40660.0,
  2099024.0,
  None,
  None,
  None]]

In [23]:

# Creating a dictionary for opening values to corresponding each day
afx_data_json = afx_json['dataset']['data']

opening = [row[1] for row in afx_data_json if row[1] != None]
print("highest opening price: " + str(max(opening)))
print("lowest opening price: " + str(min(opening)))

highest opening price: 53.11
lowest opening price: 34.0


4) What was the largest change in any one day (based on High and Low price)?

In [28]:
high = [row[2] for row in afx_data_json if row[2] != None]
low = [row[3] for row in afx_data_json if row[3] != None]
change = [abs(x1 - x2) for (x1, x2) in zip(high, low)]

print ("The largest change in any one day is:" + str(max(change)))

The largest change in any one day is:2.8100000000000023


5) What was the largest change between any two days (based on Closing Price)?

In [25]:
closing = [row[4] for row in afx_data_json if row[4] != None]
closing_prev = [row[4] for row in afx_data_json if row[4] != None][1:]
change = [abs(x1 - x2) for (x1, x2) in zip(closing, closing_prev)]
print ("The largest change between two days is:" + str(max(change)))

The largest change between two days is:2.559999999999995


6) What was the average daily trading volume during this year?
7) (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

In [32]:
trading_volume = [row[6] for row in afx_data_json]
#print(len(trading_volume))
#print(sum(trading_volume))
volume_avg = sum(trading_volume) / len(trading_volume)
print ("The avarage daily trading volume 2017: " + str(round(volume_avg,2)))

The avarage daily trading volume 2017: 89124.34


In [38]:
print("The Median trading volume 2017 : % s " % (statistics.median(trading_volume))) 

The Median trading volume 2017 : 76286.0 
